#Chapter 7   Data cleaning, merge, transformation

In [28]:
import pandas as pd
import numpy as np

##Combining and merging
+ `pd.merge(df1, df2, on=[keys] , how= , suffixes=('_left','_right'), left_index=True, sort=True)` - same as JOIN for SQL  
+ `df1.join(df2, how= , on=)` - merge by index  
+ `pd.concat([list of obj], axis= , join='inner', join_axes=[], keys=[], names=[], ignore_index=True)` - stack objects along specified axis  
+ `df1.combine_first(df2)`  - using overlapping data in df2 to fill missing values in df1, similar to `np.where`

In [15]:
df1 = pd.DataFrame({'key':list('bbacaab'), 'data1':range(7)})
df2 = pd.DataFrame({'key':list('abd'), 'data2':range(3)})
# many-to-one merge
pd.merge(df1, df2, on='key', how='outer')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,3,c,NaN
7,NaN,d,2


In [14]:
df1 = pd.DataFrame({'lkey':list('bbacaab'), 'data1':range(7)})
df2 = pd.DataFrame({'rkey':list('abd'), 'data2':range(3)})
pd.merge(df1, df2, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [18]:
# many-to-many merge
df1 = pd.DataFrame({'key':list('bbacab'), 'data1':range(6)})
df2 = pd.DataFrame({'key':list('ababd'), 'data1':range(5)})

In [19]:
pd.merge(df1, df2, on='key', how='left')

,data1_x,key,data1_y
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,2,a,0
5,2,a,2
6,3,c,NaN
7,4,a,0
8,4,a,2
9,5,b,1


In [21]:
pd.merge(df1, df2, on='key', how='inner', sort=True)

,data1_x,key,data1_y
0,2,a,0
1,2,a,2
2,4,a,0
3,4,a,2
4,0,b,1
5,0,b,3
6,1,b,1
7,1,b,3
8,5,b,1
9,5,b,3


In [25]:
left1 = df1
right1 = pd.DataFrame({'group_val':[3.5,7]}, index=['a','b'])
pd.merge(left1, right1, left_on='key', right_index=True)

,data1,key,group_val
0,0,b,7.0
1,1,b,7.0
5,5,b,7.0
2,2,a,3.5
4,4,a,3.5


In [26]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,data1,key,group_val
0,0,b,7.0
1,1,b,7.0
5,5,b,7.0
2,2,a,3.5
4,4,a,3.5
3,3,c,NaN


In [35]:
left = pd.DataFrame({'key1':list('abcde'), 'key2':range(5), 'data': np.random.randint(5)})
right = pd.DataFrame(np.arange(12).reshape((6,2)), index=[list('aabbdd'), list('223342')], columns=['event1','event2'])

In [42]:
right.reset_index()

,level_0,level_1,event1,event2
0,a,2,0,1
1,a,2,2,3
2,b,3,4,5
3,b,3,6,7
4,d,4,8,9
5,d,2,10,11


In [44]:
left1.join(right1, on='key')

,data1,key,group_val
0,0,b,7.0
1,1,b,7.0
2,2,a,3.5
3,3,c,NaN
4,4,a,3.5
5,5,b,7.0


In [50]:
s1 = pd.Series([0,1], index=['a','b'])
s2 = pd.Series([2,3,4], index=['c','d','e'])
s3 = pd.Series([5,6], index=['f','g'])
result = pd.concat([s1, s2, s3],keys=['one', 'two', 'three']) # keys here are more like labels?
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [52]:
result.unstack()

,a,b,c,d,e,f,g
one,0,1,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2,3,4,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5,6


In [53]:
pd.concat([s1, s2, s3],keys=['one', 'two', 'three'], axis=1)

,one,two,three
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [55]:
# combine_first is similar to np.where
df1 = pd.DataFrame({'a':[1, np.nan, 5, np.nan], 'b':[np.nan, 2, np.nan, 6], 'c':range(2, 18, 4)})
df1

,a,b,c
0,1,NaN,2
1,NaN,2,6
2,5,NaN,10
3,NaN,6,14


In [58]:
df2 = pd.DataFrame({'a':[5, 4, np.nan, 3, 7], 'b':[np.nan, 3, 4, 6, 8]})
df2

,a,b
0,5,NaN
1,4,3
2,NaN,4
3,3,6
4,7,8


In [59]:
df1.combine_first(df2)

,a,b,c
0,1,NaN,2
1,4,2,6
2,5,4,10
3,3,6,14
4,7,8,NaN


##Reshaping, pivoting

+ `pd.stack()` - wide to long
+ `pd.unstack(index col#/'col name', dropna=False)` - long to wide  
+ `df.pivot('target_row', 'target_col', value(if not specified then all))` - long to wide, shortcut for `set_index` and `unstack` 

In [65]:
data = pd.DataFrame(np.arange(6).reshape(2,3), index=pd.Index(['Ohio', 'Colorado'], name='state'),
                   columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [66]:
data.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [67]:
data.stack().unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [68]:
data.stack().unstack(1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [74]:
result = data.stack()
df = pd.DataFrame({'left':result, 'right':result+5}, columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [75]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [76]:
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

##Transformation

+ `df.duplicated()` - return boolean array  
+ `df.drop_duplicates([col lables, if not specified, all], keep_last=True)` - drop 
+ `df.map(function)` - map the function to the elements in the series - element-wise  
+ `obj.replace([original], [target])` or `obj.replace(dict)` - replace certain values  
+ `df.rename(index= list OR dict , columns= , in_plane=True)` - change the index and columns  
+ 

In [80]:
data = pd.DataFrame({'food':['bacon', 'pulled pork', 'bacon', 'pastrami', 'corned beef', 'Bacon', 'Pastrami', 
                             'honey ham', 'nova lox'], 'onces':[4,3,12,6,7.5,8,3,5,6]})

In [83]:
# mapping
meal_to_animal = {'bacon':'pig',
                 'pulled pork':'pig',
                 'pastrami':'cow',
                 'corned beef':'cow',
                 'honey ham':'pig',
                 'nova lox':'salmon'}

In [85]:
data['animal'] = data['food'].map(str.lower).map(meal_to_animal)
data

,food,onces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,Pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [89]:
data['food'].map(lambda x: meal_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

**Renaming axis**

In [97]:
data.index = data.index.astype(str).map(str.upper)

In [101]:
data.rename(index=str.title, columns=str.upper)

,FOOD,ONCES,ANIMAL
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,Pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [102]:
data.rename(columns={'animal':'origin'})

,food,onces,origin
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,Pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [100]:
data

,food,onces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,Pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon
